In [13]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer
from medkit.text.context import FamilyDetector
from unidecode import unidecode
import os
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)

## PREPROCESSING

In [14]:
def preprocessing(text):
    
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)  # Remplace "n°" par "numero"
    text = re.sub(r'/d°', 'deg', text)  # Remplace "/d°" par "deg"

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)  # Convertit les caractères Unicode en ASCII

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    ascii_text = re.sub(r'\s+', ' ', ascii_text)  # Remplace plusieurs espaces par un seul espace
    return ascii_text

In [54]:
text = "Il n’y avait pas d’habitudes alcoolo-tabagiques. "

In [55]:
preprocessing(text)

"Il n'y avait pas d'habitudes alcoolo-tabagiques. "

In [56]:
doc = TextDocument(text=preprocessing(text))

In [57]:
doc

TextDocument(uid='63fb6b22-175c-11ee-b6b9-59f8d42d3c26', anns=TextAnnotationContainer(doc_id='63fb6b22-175c-11ee-b6b9-59f8d42d3c26', anns=[]), metadata={}, raw_segment=Segment(uid='0d4fa714-25e2-c1a9-ec82-de8e369c51fd', label='RAW_TEXT', attrs=AttributeContainer(ann_id='0d4fa714-25e2-c1a9-ec82-de8e369c51fd', attrs=[]), metadata={}, keys=set(), spans=[Span(start=0, end=49)], text="Il n'y avait pas d'habitudes alcoolo-tabagiques. "))

In [74]:
### QUELQUES REGEX RECHERCHE ENTITES
regexp_rules_tabac = [
    RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="tabagisme", exclusion_regexp ="en bout de cigare"),
    RegexpMatcherRule(regexp=r"\bfume(e)?\b", label="tabagisme", exclusion_regexp = "residu(s)?/s+de/s+fumee(s)?"),
    RegexpMatcherRule(regexp=r"taba(c|gisme|gique)(s)?", label="tabagisme"),
    RegexpMatcherRule(regexp=r"fumeur|fumeuse", label="tabagisme"),
    RegexpMatcherRule(regexp=r"fumait", label="tabagisme"),

]
regexp_rules_alcool = [
    RegexpMatcherRule(regexp=r"\balcool(o)?", label="alcool"),
    RegexpMatcherRule(regexp=r"\bboit\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\bbuvait\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\balcoolique\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\balcoolisme\b", label="alcool"),
]

regexp_rules_familial = [
    RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bcelibataire\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bveuf\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bveuve\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bconcubinage\b", label="situation"),
    RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="situation"),
]

## DECOUPAGE

In [75]:
sent_tokenizer = SentenceTokenizer(
    output_label="sentence",
    punct_chars=[".", "?", "!"],
)
sentences = sent_tokenizer.run([doc.raw_segment])


# On sépare les phrases si il y'a "mais" "et"
synt_tokenizer = SyntagmaTokenizer(
    output_label="sentence",
    separators=[r"\bmais\b", r"\bet\b"],
)
syntagmas = synt_tokenizer.run(sentences)

## Negdetector

In [76]:
def neg_detector_tabac():
    neg_rules = [       
    NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend|fume)\s*pas"),
    NegationDetectorRule(regexp=r"jamais"),
    NegationDetectorRule(regexp=r"\bni\b"),
    NegationDetectorRule(regexp=r"\bnon\s+\b"),
    NegationDetectorRule(regexp=r"Tabac\s*[=:]?\s*0"),
    NegationDetectorRule(regexp=r"(pas|ni|ou)\s+de\s+(consommation\s+de\s+)?taba"),
    NegationDetectorRule(regexp=r"pas\s+d\'intoxication\s+tabagi"),
    NegationDetectorRule(regexp=r"0 tabac"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

def neg_detector_alcool():
    neg_rules = [
        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bne/s*consomme/s*pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
        NegationDetectorRule(regexp=r"\bpas\b"),
        NegationDetectorRule(regexp=r"\becarte\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

def neg_detector_situation_familiale():
    neg_rules = [
        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

## RECHERCHE ENTITIES

In [77]:

# Création de l'objet neg detector
neg_detector_tabac_obj = neg_detector_tabac()
neg_detector_alcool_obj = neg_detector_alcool()
neg_detector_statut_familial_obj = neg_detector_situation_familiale()

# On applique neg detector aux syntagmas
neg_detector_tabac_obj.run(syntagmas)
neg_detector_alcool_obj.run(syntagmas)
neg_detector_statut_familial_obj.run(syntagmas)

# On crée l'objet family detector
family_detector = FamilyDetector(output_label='other_detected')

# CREATION OF ENTITIES
regexp_matcher_tabac = RegexpMatcher(rules=regexp_rules_tabac, attrs_to_copy=["is_negated", "other_detected"])
regexp_matcher_alcool = RegexpMatcher(rules=regexp_rules_alcool, attrs_to_copy=["is_negated", "other_detected"])
regexp_matcher_familial = RegexpMatcher(rules=regexp_rules_familial, attrs_to_copy=["is_negated", "other_detected"])

entities_tabac = regexp_matcher_tabac.run(syntagmas)
entities_alcool = regexp_matcher_alcool.run(syntagmas)
entities_familial = regexp_matcher_familial.run(syntagmas)
family_detector.run(syntagmas)

for entity in entities_tabac:
    doc.anns.add((entity))

for entity in entities_alcool:
    doc.anns.add((entity))

for entity in entities_familial:
    doc.anns.add((entity))
print("Fin")         

Fin


## EXTRACTION STATUT TABAC

In [78]:
value_is_negated=False
value_other_detected=False
tabagisme = "UNKNOWN"
n_oui = 0
n_non = 0

for ann in doc.anns:
    for attr in ann.attrs:
        if ann.label == "tabagisme":
            print(f"text={entity.text!r}, spans={entity.spans}, label={entity.label}\n")
            if attr.label == "is_negated":
                value_is_negated = attr.value
            elif attr.label == "other_detected":
                value_other_detected = attr.value

            if value_other_detected:
                continue
            else:
                if value_is_negated:
                    n_non += 1
                else:
                    n_oui += 1
            break  # On sort de la boucle après avoir récupéré la première valeur

print(n_non)
print(n_oui)
if n_non > 0 and n_oui > 0:
    tabagisme = "FUMEUR"
elif n_non > 0:
    tabagisme = "NON-FUMEUR"
elif n_oui > 0:
    tabagisme = "FUMEUR"
tabagisme

text='alcoolo', spans=[Span(start=29, end=36)], label=alcool

0
1


'FUMEUR'

In [63]:
value_is_negated=False
value_other_detected=False
alcool = "UNKNOWN"
n_oui = 0
n_non = 0
value_is_negated = False
value_other_detected = False
count = 0  # Variable de compteur pour suivre le nombre d'occurrences

for ann in doc.anns:
    for attr in ann.attrs:
        if ann.label == "alcool":
            if attr.label == "is_negated":
                value_is_negated = attr.value
            elif attr.label == "other_detected":
                value_other_detected = attr.value
            count += 1  # Incrémenter le compteur

            if count == 2:  # Vérifier si le compteur atteint 2
                break  # Sortir de la boucle interne une fois que la deuxième valeur est récupérée

    if count == 2:  # Sortir de la boucle externe une fois que la deuxième valeur est récupérée
        if value_other_detected:
            continue
        else:
            if value_is_negated:
                n_non += 1
            else:
                n_oui += 1

        if n_non > 0 and n_oui > 0:
            alcool = "ALCOOLIQUE"
        elif n_non > 0:
            alcool = "NON-ALCOOLIQUE"
        elif n_oui > 0:
            alcool = "ALCOOLIQUE"
        break

print(alcool)

NON-ALCOOLIQUE


In [28]:
value_is_negated=False
value_other_detected=False
## Initialisation
situation = "UNKNOWN"
count = 0  # Variable de compteur pour suivre le nombre d'occurrences

# On parcourt le dico pour analyser chaque annotation trouvée
for ann in doc.anns:
    if ann.label == "situation":
        for attr in ann.attrs:
            if attr.label == "is_negated":
                value_is_negated = attr.value
            elif attr.label == "other_detected":
                value_other_detected = attr.value

        # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
        # on passe à l'annotation suivante
        if value_other_detected == True:
            continue
        else:
            situation = ann.text.lower()
            ## NORMALISATION: Seul, pas seul ou inconnu
            if re.search(r"\bmarie[e]?\b", situation):
                situation = "PAS SEUL"
            elif re.search(r"\bcelibataire\b", situation):
                situation = "SEUL"
            elif re.search(r"\bdivorce[e]?\b", situation):
                situation = "SEUL"
            elif re.search(r"\bveuf\b", situation):
                situation = "SEUL"
            elif re.search(r"\bveuve\b", situation):
                situation = "SEUL"
            elif re.search(r"\bpacse[e][s]?\b", situation):
                situation = "PAS SEUL"
            elif re.search(r"\bconcubinage\b", situation):
                situation = "PAS SEUL"
            elif re.search(r"\b(vit|habite)\sseul(e)?\b", situation):
                situation = "SEUL"

            # Si il y a une négation
            if value_is_negated == True:
                # On inverse statut_marital
                if situation == "SEUL":
                    situation = "PAS SEUL"
                else:
                    situation = "SEUL"

            if count == 3:  # Sortir de la boucle externe une fois que la troisième valeur est récupérée
                break
print(situation)

SEUL


In [224]:
#print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")
